# Импорт необходимых библиотек и модулей / Import necessary libraries and modules

In [ ]:
# Общие библиотеки / General Libraries
import pandas as pd 
import numpy as np
import plotly.graph_objects as go
from collections import defaultdict

# Pipesim библиотеки / Pipesim Libraries
from sixgill.pipesim import Model
from sixgill.definitions import Parameters, SystemVariables, ProfileVariables, Constants, Units

In [ ]:
# Unifloc VBA библиотеки и настройка / Unifloc VBA Libraries and Setup
unifloc_path = r'D:\Github\unifloc_vba' # прописываем путь к репозиторию Unifloc VBA
unifloc_xlam = unifloc_path + r'\UniflocVBA_7.xlam'
print('Путь к надстройке Unifloc VBA: '+ unifloc_xlam)

import sys
sys.path.insert(0, unifloc_path) # добавим в путь поиска пакетов python папку, где находится репозиторий Unifloc VBA
import unifloc_vba_python_api.python_api as unifloc # импортируем python_api для работы с Unifloc VBA

unf = unifloc.API(unifloc_xlam) # создаем объект имеющий доступ к расчетным модулям Unifloc VBA

print('Объект unf обеспечивает доступ к API Unifloc VBA.')

Сперва закрыть открытые Excel файлы, если таковые имеются


# Задание расположения моделей

In [ ]:
# Установим путь к модели PIPESIM для расчета вязкости, ГС, объемного коэф., плотности
MODEL_PATH = r"C:\....pips"
# Установим путь к модели PIPESIM для расчета p_dis, в качастве флюида используется вода (используется модель ESP_P_DIS_water.pips)
PIPESIM_ESP_P_DIS_WATER_MODEL_PATH  = r"C:\....pips"
# Установим путь к базовой модели для расчета p_dis, в качастве флюида используется нефть (используется модель ESP_P_DIS_oil.pips)
# Это базовая модель, в которой будем проводить изменения
PIPESIM_MODEL_PATH_ESP_P_DIS_OIL_BASE = r"C:\....pips"
# Установим путь к папке для сохранения моделей при расчете p_dis, в качастве флюида используется нефть
# В этой директории будет сохранена модель для каждого GOR в цикле
MODEL_FOLDER_PATH_ESP_P_DIS_OIL = r"C:\..."

# Сравнение результатов расчета вязкости в PIPESIM и Unifloc VBA

В этом разделе мы сравниваем результаты расчета вязкости нефти, полученные с использованием заложенных корреляций в моделях PIPESIM и Unifloc VBA.

## Моделирование в PIPESIM / Modeling in PIPESIM

In [ ]:
# Open the model
model = Model.open(MODEL_PATH, Units.METRIC)

model.set_value(context="Unifloc", 
                parameter = Parameters.BlackOilFluid.SinglePointCalibration.BUBBLEPOINTSATGAS_VALUE, 
                value=100)

studyptsim = model.tasks.ptprofilesimulation.get_conditions("Well",
            "Study 1")

# print(studyptsim)
# print(model.get_values(context="Unifloc", show_units=True))

# system_variables = [
#                     SystemVariables.PRESSURE, 
#                     SystemVariables.TEMPERATURE,
# ]

profile_variables = [
                        ProfileVariables.TEMPERATURE,
                        ProfileVariables.PRESSURE,
                        # Profilevariables.ELEVATION,
                        # Profilevariables. TOTAL_ DISTANCE,
                        ProfileVariables.VISCOSITY_OIL_INSITU, 
                        ProfileVariables.SOLUTION_GAS_IN_OIL_INSITU,
                        ProfileVariables.FORMATION_VOLUME_FACTOR_OIL_INSITU,
                        ProfileVariables.DENSITY_OIL_INSITU,
]

results = model.tasks.ptprofilesimulation.run(
            producer="Well",
            profile_variables=profile_variables, 
#             system_variables=system_variables
)

# print(results-profile)

pressure_data = results.profile[next(iter(results.profile.keys()))][ProfileVariables.PRESSURE] # bara
viscosity_data = results.profile[next(iter(results.profile.keys()))][ProfileVariables.VISCOSITY_OIL_INSITU] # cP
temperature_data = results.profile[next(iter(results.profile.keys()))][ProfileVariables.TEMPERATURE] # C
solution_gas_data = results.profile[next(iter(results.profile.keys()))][ProfileVariables.SOLUTION_GAS_IN_OIL_INSITU]
fvf_data = results.profile[next(iter(results.profile.keys()))][ProfileVariables.FORMATION_VOLUME_FACTOR_OIL_INSITU]
density_data = results.profile[next(iter(results.profile.keys()))][ProfileVariables.DENSITY_OIL_INSITU]

print("Данные из модели PIPESIM получены.")

## Моделирование в Unifloc VBA / Modeling in Unifloc VBA

In [ ]:
# задаем флюид
fluid = unf.encode_PVT(gamma_gas=0.6, 
                       gamma_oil=0.86, 
                       gamma_wat=1.1, 
                       rsb_m3m3=100, 
                       pb_atma=130, 
                       t_res_C=80, 
                       bob_m3m3=1.2, 
                       muob_cP=0.6, 
                       PVT_corr_set=0)

# выведем итоговую json строку
print(fluid)

In [ ]:
temperature_unf = 30 # значение температуры такое же, как и в модели PIPESIM
calculated_unf_viscosity = [unf.PVT_mu_oil_cP(p*0.9869234, temperature_unf, fluid) for p in pressure_data] # добавлен перевод (bara) в (atma) для корректного расчета в Unifloc

## Распределение температуры в PIPESIM

В данном разделе убеждаемся, что температура, использованная в модели PIPESIM, остается постоянной.

In [ ]:
# Определение данных температуры из модели PIPESIM
trace_pipesim_temperature = go.Scatter(x=pressure_data, 
                                       y=temperature_data, 
                                       mode='lines+markers', 
                                       name='PIPESIM Temperature')

# Создание макета графика для температуры
layout_temperature = go.Layout(title='Pressure vs Temperature', 
                               xaxis=dict(title='Pressure (bara)'), 
                               yaxis=dict(title='Temperature (C)'))


fig_temperature = go.Figure(data=trace_pipesim_temperature, layout=layout_temperature)
fig_temperature.show()

## Сравнение результатов расчета вязкости / Comparison of Viscosity Calculation Results

In [ ]:
# Определение графиков / Define the traces
trace_pipesim = go.Scatter(x=pressure_data, y=viscosity_data, mode='lines+markers', name='PIPESIM Viscosity')
trace_unifloc = go.Scatter(x=pressure_data, y=calculated_unf_viscosity, mode='lines+markers', name='Unifloc Viscosity (PVT_mu_oil_cP)')

# Создание макета для графика / Create the layout for the plot
layout = go.Layout(
    title='Pressure vs Viscosity Comparison',
    xaxis=dict(title='Pressure (bara)'),
    yaxis=dict(title='Viscosity (cP)')
)

# Создание графика / Create the figure
fig = go.Figure(data=[trace_pipesim, trace_unifloc], layout=layout)

# Отображение графика / Display the plot
fig.show()

# Сравнение результатов расчета газосодержания в PIPESIM и Unifloc VBA

In [ ]:
# Расчет данных для газосодержания с использованием соответствующей функции в Unifloc
calculated_solution_gas_unifloc = [unf.PVT_rs_m3m3(p*0.9869234, temperature_unf, fluid)  for p in pressure_data]

# Определение данных газосодержания из модели PIPESIM
trace_pipesim_solution_gas = go.Scatter(x=pressure_data, y=solution_gas_data, mode='lines+markers', name='Solution Gas in PIPESIM')

# Определение расчетных данных газосодержания из Unifloc
trace_unifloc_solution_gas = go.Scatter(x=pressure_data, y=calculated_solution_gas_unifloc, mode='lines+markers', name='Solution Gas in Unifloc')

# Создание макета графика для газосодержания
layout_solution_gas = go.Layout(title='Pressure vs Solution Gas', xaxis=dict(title='Pressure (bara)'), yaxis=dict(title='Solution Gas (sm3/sm3)'))

# Создание графика для газосодержания
fig_solution_gas = go.Figure(data=[trace_pipesim_solution_gas, trace_unifloc_solution_gas], layout=layout_solution_gas)
fig_solution_gas.show()

# Сравнение результатов расчета газосодержания в PIPESIM и Unifloc VBA

In [ ]:
# Расчет данных для объемного коэффициента с использованием соответствующей функции в Unifloc
calculated_fvf_unifloc = [unf.PVT_bo_m3m3(p*0.9869234, temperature_unf, fluid) for p in pressure_data]

# Определение данных объемного коэффициента из модели PIPESIM
trace_pipesim_fvf = go.Scatter(x=pressure_data, y=fvf_data, mode='lines+markers', name='FVF in PIPESIM')

# Определение расчетных данных объемного коэффициента из Unifloc
trace_unifloc_fvf = go.Scatter(x=pressure_data, y=calculated_fvf_unifloc, mode='lines+markers', name='FVF in Unifloc')

# Создание макета графика для объемного коэффициента
layout_fvf = go.Layout(title='Pressure vs FVF', xaxis=dict(title='Pressure (bara)'), yaxis=dict(title='FVF'))

# Создание графика для объемного коэффициента
fig_fvf = go.Figure(data=[trace_pipesim_fvf, trace_unifloc_fvf], layout=layout_fvf)
fig_fvf.show()

# Сравнение результатов расчета вязкости в PIPESIM и Unifloc VBA

In [ ]:
# Расчет данных для плотности с использованием соответствующей функции в Unifloc
calculated_density_unifloc = [unf.PVT_rho_oil_kgm3(p*0.9869234, temperature_unf, fluid) for p in pressure_data]

# Определение данных плотности из модели PIPESIM
trace_pipesim_density = go.Scatter(x=pressure_data, y=density_data, mode='lines+markers', name='Density in PIPESIM')

# Определение расчетных данных плотности из Unifloc
trace_unifloc_density = go.Scatter(x=pressure_data, y=calculated_density_unifloc, mode='lines+markers', name='Density in Unifloc')

# Создание макета графика для плотности
layout_density = go.Layout(title='Pressure vs Density', xaxis=dict(title='Pressure (bara)'), yaxis=dict(title='Density (kg/m3)'))

# Создание графика для плотности
fig_density = go.Figure(data=[trace_pipesim_density, trace_unifloc_density], layout=layout_density)
fig_density.show()

# Сравнение расчета давления на выходе ЭЦН в PIPESIM и Unifloc / Comparison of ESP Discharge Pressure Calculation in PIPESIM and Unifloc

Этот раздел предназначен для сравнения расчета давления на выходе ЭЦН с использованием двух различных программных инструментов: PIPESIM и Unifloc. В качестве флюида используется вода.

In [ ]:
import random

In [ ]:
#---------------------------------------------------------------
# PIPESIM calculation
#---------------------------------------------------------------
# Откроем модель PIPESIM
pipesim_model = Model.open(PIPESIM_ESP_P_DIS_WATER_MODEL_PATH, Units.METRIC)
print(pipesim_model.about())

# Список переменных профиля для PIPESIM
pipesim_profile_variables = [
    ProfileVariables.TEMPERATURE,
    ProfileVariables.PRESSURE,
    ProfileVariables.ELEVATION,
    ProfileVariables.TOTAL_DISTANCE,
]

# Диапазон расхода жидкости для Source 1 (PIPESIM)
pipesim_liquid_flow_rates = list(range(5, 300, 20))

# Список для хранения значений давления для Source 1 (PIPESIM)
pipesim_pressures = []

# Рассчет давлений для Source 1 (PIPESIM)
for liquid_flow_rate in pipesim_liquid_flow_rates:
    parameters = {
        Parameters.PTProfileSimulation.INLETPRESSURE: 151.9875,  # 150 (atma) from Unifloc converted to (bara) in PIPESIM
        Parameters.PTProfileSimulation.LIQUIDFLOWRATE: liquid_flow_rate,
        Parameters.PTProfileSimulation.FLOWRATETYPE: Constants.FlowRateType.LIQUIDFLOWRATE,
        Parameters.PTProfileSimulation.CALCULATEDVARIABLE: Constants.CalculatedVariable.OUTLETPRESSURE,
    }

    # Запуск симуляции и получение результатов
    print("Running PT profile simulation in PIPESIM")
    results = pipesim_model.tasks.ptprofilesimulation.run(
        producer="Well",
        parameters=parameters,
#         system_variables=pipesim_system_variables,
        profile_variables=pipesim_profile_variables
    )

    # Получение давления из профиля
    for case, profile in results.profile.items():
#         pressure = profile[ProfileVariables.PRESSURE]  # Пример получения давления
#         pipesim_pressures.append(pressure)
        profile_df = pd.DataFrame.from_dict(profile)
        pressure = profile_df[profile_df["BranchEquipment"] == 'ESP']["Pressure"].iloc[0]*0.986923 # converted to (atma)

    # Append pressure value to the list
    pipesim_pressures.append(pressure)

    
#---------------------------------------------------------------
# Unifloc calculation
#---------------------------------------------------------------
# Диапазон расхода жидкости для Source 2 (Unifloc)
unifloc_liquid_flow_rates = pipesim_liquid_flow_rates.copy()

# Параметры PVT для модели ЭЦН
gamma_gas_value = 0.6
gamma_oil_value = 0.86
gamma_wat_value = 1.02
rsb_m3m3_value = 120
pb_atma_value = 130
t_res_C_value = 80
bob_m3m3_value = 1.2
muob_cP_value = 0.6
PVT_corr_set_value = 0

# Создание модели ЭЦН с использованием Unifloc
esp_params = unf.encode_ESP_pump(
    q_nom_sm3day=120,
    head_nom_m=2000,
    freq_nom_Hz=50,
    num_stages=10,
    calibr_head=1,
    calibr_rate=1,
    calibr_power=1,
    gas_correct_model=1,
    gas_correct_stage_by_stage=0,
    dnum_stages_integrate=1
)


fig = go.Figure()

# Выбор случайного цвета
random_color = f'rgb({random.randint(0, 255)}, {random.randint(0, 255)}, {random.randint(0, 255)})'

# Рассчет давления p_dis для Source 2 (Unifloc)
unifloc_p_dis_values = []

for q_liq_value in unifloc_liquid_flow_rates:
    # Создание PVT_ESP с использованием заданных параметров
    PVT_ESP = unf.encode_PVT(
        gamma_gas=gamma_gas_value,
        gamma_oil=gamma_oil_value,
        gamma_wat=gamma_wat_value,
        rsb_m3m3=rsb_m3m3_value,
        pb_atma=pb_atma_value,
        t_res_C=t_res_C_value,
        bob_m3m3=bob_m3m3_value,
        muob_cP=muob_cP_value,
        PVT_corr_set=PVT_corr_set_value
    )

    # Создание feed_esp для расчета p_dis
    feed_esp = unf.encode_feed(
        q_liq_sm3day=q_liq_value,
        fw_perc=100,  # Процентная вода
        rp_m3m3=120,
        q_gas_free_sm3day=0,
        fluid=PVT_ESP
    )

    # Расчет p_dis с использованием модели ЭЦН из Unifloc
    p_dis = unf.ESP_p_atma(
        p_calc_atma=150,  # Расчетное давление
        t_intake_C=80,    # Температура на входе
        t_dis_C=80,       # Температура на выходе
        feed=feed_esp,
        esp_json=esp_params,
        freq_Hz=48.5,
        calc_along_flow=1,
        param=1,
        h_mes_top=2000
    )

    # Добавление рассчитанного значения p_dis в список
    unifloc_p_dis_values.append(p_dis)


# Создание DataFrame для удобства визуализации
df = pd.DataFrame({
    'liquid_flow_rates_source1': pipesim_liquid_flow_rates,
    'pressures_source1': pipesim_pressures,
    'liquid_flow_rates_source2': unifloc_liquid_flow_rates,
    'p_dis_values_source2': unifloc_p_dis_values
})

# Добавление линий на график
fig.add_trace(go.Scatter(x=pipesim_liquid_flow_rates, y=pipesim_pressures, mode='lines',
                         name='Source 1 (PIPESIM)', line=dict(color=random_color)))

fig.add_trace(go.Scatter(x=unifloc_liquid_flow_rates, y=unifloc_p_dis_values, mode='lines',
                         name='Source 2 (Unifloc)', line=dict(color=random_color, dash='dash')))

# Закрытие модели PIPESIM
pipesim_model.close()

# Обновление графика
fig.update_layout(
    title='Pressure Discharge Characteristics of ESP',
    xaxis_title='Liquid Flow Rate (sm3/day)',
    yaxis_title='Pressure (atma)',
    legend_title='Source'
)

# Отображение графика
fig.show()

TO DO: перейти от значения давления на выходе к перепаду давления, чтобы избежать "лишних" перевод из одних единиц измерения в другие

# Анализ давления на выходе из УЭЦН для нефти / Analysis of ESP Discharge Pressure for Oil

Этот модуль сравнивает давление на выходе из ЭЦН при различных газосодержаниях (GOR), используя два источника: PIPESIM и Unifloc. В данном исследовании в качестве флюида используется нефть. Данные визуализируются для понимания зависимости давления от расхода жидкости при различных значениях GOR.

In [ ]:
# Open the PIPESIM model
model = Model.open(PIPESIM_MODEL_PATH_ESP_P_DIS_OIL_BASE, Units.METRIC)
print(model.about())

profile_variables = [
    ProfileVariables.TEMPERATURE,
    ProfileVariables.PRESSURE,
    ProfileVariables.ELEVATION,
    ProfileVariables.TOTAL_DISTANCE,
]

# Source 1 (PIPESIM)
gor_values_source1 = list(range(50, 150, 25))
liquid_flow_rates_source1 = list(range(5, 300, 20))
pressure_values_dict_source1 = {}

# Source 2 (Unifloc)
rp_range_source2 = gor_values_source1.copy()
q_liq_range_source2 = liquid_flow_rates_source1.copy()

# PVT parameters for ESP model
gamma_gas_value = 0.6
gamma_oil_value = 0.86
gamma_wat_value = 1.1
rsb_m3m3_value = 120
pb_atma_value = 130
t_res_C_value = 80
bob_m3m3_value = 1.2
muob_cP_value = 0.6
PVT_corr_set_value = 0

# Fluid flow parameters
q_liq_sm3day_value = 50
fw_perc_value = 0
rp_m3m3_value = None
q_gas_free_sm3day_value = 0

# ESP parameters
ESP_q_nom_value = 120
ESP_head_nom_value = 2000
freq_nom_value = 48.5
num_stages_value = 10
calibr_head_value = 1
calibr_rate_value = 1
calibr_power_value = 1 
gas_correct_model_value = 1 
gas_correct_stage_by_stage_value = 0
dnum_stages_integrate_value = 1

# Parameters for calculating p_dis
p_calc_atma_value = 150
t_intake_C_value = 80
t_dis_C_value = 80
freq_Hz_value = 50
calc_along_flow_value = 1
param_value = 1
h_mes_top_value = 2000

# Encode ESP pump
esp = unf.encode_ESP_pump(
    q_nom_sm3day=ESP_q_nom_value,
    head_nom_m=ESP_head_nom_value,
    freq_nom_Hz=freq_nom_value,
    num_stages=num_stages_value,
    calibr_head=calibr_head_value,
    calibr_rate=calibr_rate_value,
    calibr_power=calibr_power_value,
    gas_correct_model=gas_correct_model_value,
    gas_correct_stage_by_stage=gas_correct_stage_by_stage_value,
    dnum_stages_integrate=dnum_stages_integrate_value
)

# print(esp)

# Create subplots
fig = go.Figure()

used_colors = set()
all_colors = ['blue', 'green', 'red', 'cyan', 'magenta', 'yellow', 'black', 'orange', 'purple', 'brown']

# Create pairs of plots for each GOR value from both sources
for gor_source1, gor_source2 in zip(gor_values_source1, rp_range_source2):
    pressures_source1 = []  # List to store pressure values
    p_dis_values_source2 = []

    print("gor_source2", gor_source2)
    
    # Source 1 (PIPESIM)
    model.set_value(
        context="Unifloc_book_1",
        parameter=Parameters.BlackOilFluid.GOR,
        value=gor_source1
    )

    random_color = random.choice(all_colors)

    model.save(MODEL_FOLDER_PATH_ESP_P_DIS_OIL + "\ESP_GOR_" + str(gor_source1) + ".pips")
    
    # Calculate pressures for Source 1
    for liquid_flow_rate in liquid_flow_rates_source1:
        parameters = {
            Parameters.PTProfileSimulation.INLETPRESSURE: 151.9875,  # 150 (atma) Unifloc converted to (bara) PIPESIM
            Parameters.PTProfileSimulation.LIQUIDFLOWRATE: liquid_flow_rate,
            Parameters.PTProfileSimulation.FLOWRATETYPE: Constants.FlowRateType.LIQUIDFLOWRATE,
            Parameters.PTProfileSimulation.CALCULATEDVARIABLE: Constants.CalculatedVariable.OUTLETPRESSURE,
        }

        # Run the simulation and print out the results
        print("Running PT profile simulation")
        results = model.tasks.ptprofilesimulation.run(producer="Well",
                                                      parameters=parameters,
                                                      profile_variables=profile_variables)

        # Profile results
        for case, profile in results.profile.items():
            profile_df = pd.DataFrame.from_dict(profile)
            pressure = profile_df[profile_df["BranchEquipment"] == 'ESP']["Pressure"].iloc[0] * 0.986923
            print("Pressure: ", pressure)

        # Append pressure value to the list
        pressures_source1.append(pressure)

    # Calculate p_dis values for Source 2
    for q_liq_value in q_liq_range_source2:
        print("q_liq_value", q_liq_value)
        
        # Create PVT_ESP using specified parameters
        PVT_ESP = unf.encode_PVT(
            gamma_gas=gamma_gas_value,
            gamma_oil=gamma_oil_value,
            gamma_wat=gamma_wat_value,
            rsb_m3m3=rsb_m3m3_value,
            pb_atma=pb_atma_value,
            t_res_C=t_res_C_value,
            bob_m3m3=bob_m3m3_value,
            muob_cP=muob_cP_value,
            PVT_corr_set=PVT_corr_set_value
        )
        
        feed_esp = unf.encode_feed(
            q_liq_sm3day=q_liq_value,
            fw_perc=fw_perc_value,
            rp_m3m3=gor_source2,
            q_gas_free_sm3day=q_gas_free_sm3day_value,
            fluid=PVT_ESP
        )

        p_dis = unf.ESP_p_atma(
            p_calc_atma=p_calc_atma_value,
            t_intake_C=t_intake_C_value,
            t_dis_C=t_dis_C_value,
            feed=feed_esp,
            esp_json=esp,
            freq_Hz=freq_Hz_value,
            calc_along_flow=calc_along_flow_value,
            param=param_value,
            h_mes_top=h_mes_top_value
        )
        
#         print(p_dis)
        
        p_dis_values_source2.append(p_dis)
    
    # Create DataFrame
    df = pd.DataFrame({
        'liquid_flow_rates_source1': liquid_flow_rates_source1,
        'pressures_source1': pressures_source1,
        'q_liq_range_source2': q_liq_range_source2,
        'p_dis_values_source2': p_dis_values_source2
    })
    
    print("df for gor_source1:", gor_source1, "gor_source2:", gor_source2, ":")
    print(df)
    
    fig.add_trace(go.Scatter(x=liquid_flow_rates_source1, y=pressures_source1, mode='lines',
                             name=f'Source 1: GOR {gor_source1}', line=dict(color=random_color)))
    
    fig.add_trace(go.Scatter(x=q_liq_range_source2, y=p_dis_values_source2, mode='lines',
                             name=f'Source 2: GOR {gor_source2}', line=dict(color=random_color, dash='dash')))

model.close()
    
fig.update_layout(
    title='Dependence of Pressure on Liquid Flow Rate for Different GOR Values',
    xaxis_title='Liquid Flow Rate (sm3/day)',
    yaxis_title='Pressure (atma)',
    legend_title='Source'
)

# Show plot
fig.show()

TO DO: поиск расхождения

# Дополнительно. Черновик.

## Анализ зависимости Pressure vs density при разных Parameters.BlackOilFluid.DEADOILDENSITY в модели PIPESIM

In [ ]:
density_range = range(850, 866, 5)

pressure_data_list = []
density_data_list = []
traces = []

model = Model.open(MODEL_PATH, Units.METRIC)
# print(model.get_values(context="Unifloc", show_units=True))

for density_value in density_range:
    
#     print(model.get_values(context="Unifloc", show_units=True))
    
    print("Density Value is", density_value)
    
#     values = defaultdict(dict)
#     values["Unifloc"][Parameters.BlackOilFluid.DEADOILDENSITY] = density_value
#     model.set_values(values)
    
    model.set_value(context="Unifloc", 
                parameter = Parameters.BlackOilFluid.DEADOILDENSITY, 
                value=density_value)

    
    results = model.tasks.ptprofilesimulation.run(
                producer="Well",
                profile_variables=profile_variables, 
#                 system_variables=system_variables
    )

    pressure_data = results.profile[next(iter(results.profile.keys()))][ProfileVariables.PRESSURE]
    density_data = results.profile[next(iter(results.profile.keys()))][ProfileVariables.DENSITY_OIL_INSITU]

    pressure_data_list.append(pressure_data)
    density_data_list.append(density_data)
    
model.close()    

for i, density_value in enumerate(density_range):
    trace_density = go.Scatter(x=pressure_data_list[i], 
                               y=density_data_list[i], 
                               mode='lines', name=f'Density: {density_value}')
    traces.append(trace_density)

# calculated_density_source_2 = [unf.PVT_rho_oil_kgm3(p, temperature_unf, fluid) for p in pressure_data]
# trace_calculated_density_source_2 = go.Scatter(x=pressure_data, y=calculated_density_source_2, mode='lines+markers', name='Calculated Density (Source 2 = Unifloc)')
# traces.append(trace_calculated_density_source_2)
    
layout_combined = go.Layout(title='Pressure vs Density for Various Density Values', xaxis=dict(title='Pressure'), yaxis=dict(title='Density'))

figure_combined = go.Figure(data=traces, layout=layout_combined)
figure_combined.show()

## Анализ зависимости Pressure vs density при разных корреляциях в модели PIPESIM

In [ ]:
# Define correlations
correlations = {
    "BEGGSANDROBINSON" : 'BeggsAndRobinson',
    "CHEWANDCONNALY": 'ChewandConnaly',
    "KARTOATMODJO": "Kartoatmodjo",
    "KHAN": "Khan",
    "DEGHETTO": 'DeGhetto',
    "HOSSAIN": 'Hossain',
    "ELSHARKAWY": 'Elsharkawy',
    "PETROSKYEARSHAD": 'PetroskyFarshad'
}

traces = []
model = Model.open(MODEL_PATH, Units.METRIC)
# print(model.get_values(context="Unifloc", show_units=True))

for correlation_name, correlation_value in correlations.items():
    print("Correlation Is :", correlation_name)
#     values = defaultdict(dict)
#     values["Unifloc"][Parameters.BlackOilFluid.SinglePointCalibration.LIVEOILVISCCORRELATION] = correlation_value
#     model.set_values(values)
    
    model.set_value(context="Unifloc", 
            parameter = Parameters.BlackOilFluid.SinglePointCalibration.LIVEOILVISCCORRELATION, 
            value=correlation_value)
    
    results = model.tasks.ptprofilesimulation.run(
        producer="Well",
        profile_variables=profile_variables, 
#         system_variables=system_variables
    )

    pressure_data = results.profile[next(iter(results.profile.keys()))][ProfileVariables.PRESSURE]
    density_data = results.profile[next(iter(results.profile.keys()))][ProfileVariables.DENSITY_OIL_INSITU]

    trace = go.Scatter(x=pressure_data, y=density_data, mode='lines', name=f'Correlation: {correlation_name}')
    traces.append(trace)

# calculated_density_source_2 = [unf.f_PVT_rho_oil_kgm3(p, temperature_unf, fluid) for p in pressure_data]
# trace_calculated_density_source_2 = go.Scatter(
#     x=pressure_data,
#     y=calculated_density_source_2,
#     mode='lines+markers',
#     name=f'Calculated Density (Source 2 = Unifloc)'
# )
# traces.append(trace_calculated_density_source_2)

model.close()

layout_combined = go.Layout(
    title='Pressure vs Density for Various Correlations',
    xaxis=dict(title='Pressure'),
    yaxis=dict(title='Density')
)

figure_combined = go.Figure(data=traces, layout=layout_combined)
figure_combined.show()